# Hotel-Reviews-with-Gen-AI-Snowflake

## Discover the data:

In [ ]:
DESC TABLE HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS

In [ ]:
# Create a country filter for hotel reviews in Mexico
country_filter = 'Mexico'

In [ ]:
SELECT country, COUNT(*) AS num_reviews
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
-- Apply the country_filter
WHERE country = 'Mexico'
GROUP BY country

In [ ]:
SELECT country, COUNT(*) AS num_reviews
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
-- Apply the country_filter
GROUP BY country
ORDER BY num_reviews DESC
LIMIT 5;

## Summarizing a review

In [ ]:
review = """
It was the last night of our road trip and we wanted 
to treat ourselves and stay somewhere a bit nicer. 
We stayed out of season so it was affordable. 
Beautiful hotel and great service.
We called prior to arriving to ask about parking. 
They said not to worry and they would park the car when we arrived.
They didn't; however tell us parking was 45euro a night. 
That is my only criticism. We could have moved the car
but we felt a bit embarrassed so we left it there for the night (we didn't want to look cheap). 
Loved the hotel I just think they should be a bit clearer about that 
sort of cost because it really did add to the price of our stay as a couple 
who don't often frequent five-star hotels. 
Otherwise; lovely staff. We felt very welcome.
"""

In [ ]:
# Import the summarize function
from snowflake.cortex import summarize
# Summarize the review
response = summarize(text=review)
print(response)


## Text Generation

In [ ]:
SELECT DESCRIPTION, RATING
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE date = '2023-07-14'
    AND RATING < 6

In [ ]:
# Import packages
import pandas as pd
from snowflake.cortex import complete

# Convert the review to a pandas DataFrame
df = cell18.to_pandas()

# Extract the first review as a string
review = df["DESCRIPTION"].iloc[0]
print(review)

In [ ]:
prompt = f"""Write a 2-sentence response to the following review:\n---\n{review}
        \n---\nAcknowledge the feedback and outline planned actions. 
        Return only the response text."""

# Generate and print a response
response = complete(model='llama3.1-8b',
                    prompt = prompt,
                    # Set temperature and limit tokens
					options={
                    	'temperature':0.3,
                    	'max_tokens':150
                    	}
                	)
print(response)

In [ ]:
SELECT 
  DESCRIPTION,
  -- Generate a response
  AI_COMPLETE(
    'llama3.1-8b',
    -- Complete a prompt
    PROMPT('Write a polite 1-sentence response to this hotel review: {0}.', DESCRIPTION),
    -- Set temperature and limit tokens
    {'temperature':0.5,
      'max_tokens':60}
  ) AS completion
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE RATING > 8
AND COUNTRY = 'France'
ORDER BY DATE DESC
LIMIT 2

## Building a classification pipeline

In [ ]:
import pandas as pd
import json

# Import the required function
from snowflake.cortex import classify_text

categories = ["overall_experience", "location", "staff",
              "food_beverages", "facilities"]

In [ ]:
SELECT *
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE city = 'Geneva'
	AND RATING < 6
    ORDER BY DATE DESC
LIMIT 5

In [ ]:
reviews = cell8.to_pandas()


# Create a function to add one new column contaning the predicted category
def classification(text):
    # Use the relevant Snowflake Cortex function 
    result = classify_text(
        str_input=text,
        categories=categories
    )
    # Convert result into a dictionary 
    result_dict = json.loads(result)
    return result_dict['label']

In [ ]:
# Apply the classification function
reviews["category"] = reviews["DESCRIPTION"].apply(classification)

reviews["category"].value_counts()

## Sentiment analysis

In [ ]:
SELECT 
    DESCRIPTION, 
    
    -- Perform sentiment analysis
    AI_SENTIMENT(DESCRIPTION)['categories'][0]['sentiment']
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE hotel_name = 'Hyatt Regency Palais'

    -- Filter records for Sep 2023
	AND EXTRACT(MONTH FROM DATE) = 9
    AND EXTRACT(YEAR FROM DATE) = 2023
ORDER BY DATE DESC

## Translate

## Translate Spanish

In [ ]:
SELECT DESCRIPTION
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
-- Filter for Spanish reviews
WHERE LANGUAGE = 'es'
ORDER BY DATE DESC 
LIMIT 1;

In [ ]:
df = cell26.to_pandas()

review = df["DESCRIPTION"].iloc[0]

print(review)

In [ ]:
# Import the relevant function
from snowflake.cortex import translate

# Call the function and print the result
translated = translate(text=review, 
                  from_language='es', 
                  to_language='en')

print(translated)

## Translate more

In [ ]:
SELECT DESCRIPTION
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE LANGUAGE = 'fr'
-- Sort results by date from newest to oldest
ORDER BY DATE DESC
LIMIT 1;

In [ ]:
df = cell30.to_pandas()
review = df["DESCRIPTION"].iloc[0]

print(review)

In [ ]:
# Import the relevant function
from snowflake.cortex import translate
# Translate into English and German
english = translate(text=review, 
                  from_language='fr', 
                  to_language='en')

german = translate(text=english, 
                  from_language='en', 
                  to_language='de')

# Print the original, English, and German reviews
print(f"Original (French): {review}\nEnglish: {english}\nGerman: {german}")

## Answer extraction

In [ ]:
SELECT DESCRIPTION
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
-- Set the appropriate filters
WHERE HOTEL_NAME = 'Grand Fiesta Americana'
    AND DATE = '2023-07-30'
    AND RATING < 6
LIMIT 1;

In [ ]:
df = cell36.to_pandas()
review = df["DESCRIPTION"].iloc[0]

print(review)

In [ ]:
# Import the relevant function
from snowflake.cortex import extract_answer
# Find out what the main issue was
response = extract_answer(
    from_text=review,
    question='What is the main issue with this review?'
)

# Print the response
print(response)

In [ ]:
SELECT DESCRIPTION
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
-- Filter for Italian reviews
WHERE LANGUAGE = 'it'
ORDER BY DATE DESC 
LIMIT 1;

In [ ]:
df = cell34.to_pandas()
review = df["DESCRIPTION"].iloc[0]

# Import the relevant function
from snowflake.cortex import extract_answer

# Find out what the main issue was
response = extract_answer(
    from_text=review,
    question='What is this review about?'
)

# Print the response
print(response)

## Designing multi step AI workflows

### Build a review pipeline: translate, summarize, classify, response, and finally translate back to original language.

In [ ]:
SELECT DESCRIPTION
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE LANGUAGE = 'es'
ORDER BY DATE
LIMIT 1;

In [ ]:
df = cell45.to_pandas()
review = df['DESCRIPTION'].iloc[0]

print(review)

In [ ]:
from snowflake.cortex import translate, summarize, complete

# Translate the review into English
translated = translate(
    text=review,
    from_language='es',
    to_language='en'
)

# Summarize the English review
summarized = summarize(
    text=translated
)
print(summarized)

In [ ]:
# Create categories
categories = ['staff', 'location', 'cleanliness', 'food and drink']

# Classify the summarized review
review_category = classify_text(
    str_input=summarized,
    categories=categories
)

# Print the category
print(review_category)

In [ ]:
# Generate a response
response = complete(
    model='llama3.1-8b',
    prompt=f"""Write a short, professional response to the following review:\n---\n{summarized}.
               \n---\nReturn only the response text.""",
    options={
        'temperature': 0.2,
        'max_tokens': 100
    }
)

# Translate the response into Spanish
spanish_response = translate(
    text=response,
    from_language='en',
    to_language='es'
)

print(spanish_response)

In [ ]:
SELECT 
    DESCRIPTION, RATING,
    
    -- Perform sentiment analysis
    AI_SENTIMENT(DESCRIPTION)['categories'][0]['sentiment']
FROM HOTEL_REVIEWS.PUBLIC.HOTEL_REVIEWS
WHERE country = 'Switzerland'

    -- Filter records for Sep 2023
	--AND EXTRACT(MONTH FROM DATE) = 9
    --AND EXTRACT(YEAR FROM DATE) = 2023
ORDER BY DATE DESC
LIMIT 10;


Use the code below to visualize the data in dataframe:

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")